In [1]:
# Importing Required Packages.

In [2]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler
from textblob import TextBlob

In [3]:
#Twitter App Credentials

In [4]:
consumer_key = "sMeKVRgv0BidivpNT5B7kpIF5"
consumer_secret = "r7dCpljH8kJivSlJ9fHwuH31ElXoQGPxC4Laz1z09sXkRmZ7EC"
access_token = "3194988066-uKvAfRk77momZwJgkDaVSpmfvi4AlUpklnTGCye"
access_token_secret = "9U3WbKJbXB5P1lEA1wC7lJbFCK0iOb2mXKsw6akyOePBN"

In [5]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [6]:
query = "Covid cases in India"

In [7]:
# Searching tweets using search function
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x0000021C9AE36FC8>, _json={'created_at': 'Tue Nov 03 13:08:23 +0000 2020', 'id': 1323612991690924034, 'id_str': '1323612991690924034', 'full_text': "Tracking #COVID19 | November 3, 2020 (6:30 pm, IST)\n\nIndia's total cases reach 8,267,623\n\nFor complete coverage on Covid-19 outbreak, log on to: https://t.co/XKhgwwvvgU https://t.co/f6TpuEXIiS", 'truncated': False, 'display_text_range': [0, 168], 'entities': {'hashtags': [{'text': 'COVID19', 'indices': [9, 17]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/XKhgwwvvgU', 'expanded_url': 'http://toi.in/coronavirus', 'display_url': 'toi.in/coronavirus', 'indices': [145, 168]}], 'media': [{'id': 1323612896119549954, 'id_str': '1323612896119549954', 'indices': [169, 192], 'media_url': 'http://pbs.twimg.com/media/El5q6o1U8AIIq_D.jpg', 'media_url_https': 'https://pbs.twimg.com/media/El5q6o1U8AIIq_D.jpg', 'url': 'https://t.co/f6TpuEXIiS', 'display_url': 'pic.twitter.com/f6TpuEXIiS', 

In [8]:
# Creating a Dataframe for storing the Tweets.
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [9]:
# Searching Tweets and saving them in the DataFrame.
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, exclude = 'retweets', lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 2000:
            break
        else:
            pass

In [10]:
stream(query , file_name='my_tweets')

In [11]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,"Tracking #COVID19 | November 3, 2020 (6:30 pm,...",The Times Of India,610027,13204586,New Delhi,True,26,1,2020-11-03 13:08:23
1,Daily COVID 19 Cases &amp; Deaths Decline In I...,Mathrubhumi News,46861,50696,"Kerala, India",True,1,0,2020-11-03 13:03:02
2,"Active #COVID__19 cases in India — 539,431\n\n...",ORF,156349,80297,India,True,5,3,2020-11-03 13:00:42
3,COVID 19 cases in India until : 03/11/2020 18:...,Coronavirus Bot 🇮🇳,2862,116,,False,0,0,2020-11-03 13:00:00
4,Explaining India's miraculous Covid recovery\n...,NEWS9,93254,26271,BENGALURU,True,1,0,2020-11-03 12:57:30


In [12]:
# Cleaning the Tweets.
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [13]:
# Analyzing the Tweets.
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [14]:
# Adding the Cleaned Tweets to the Dataframe.
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [15]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,"Tracking #COVID19 | November 3, 2020 (6:30 pm,...",The Times Of India,610027,13204586,New Delhi,True,26,1,2020-11-03 13:08:23,Tracking November 3 2020 6 30 pm IST India s t...
1,Daily COVID 19 Cases &amp; Deaths Decline In I...,Mathrubhumi News,46861,50696,"Kerala, India",True,1,0,2020-11-03 13:03:02,Daily COVID 19 Cases amp Deaths Decline In India
2,"Active #COVID__19 cases in India — 539,431\n\n...",ORF,156349,80297,India,True,5,3,2020-11-03 13:00:42,Active 19 cases in India 539 431
3,COVID 19 cases in India until : 03/11/2020 18:...,Coronavirus Bot 🇮🇳,2862,116,,False,0,0,2020-11-03 13:00:00,COVID 19 cases in India until 03 11 2020 18 30...
4,Explaining India's miraculous Covid recovery\n...,NEWS9,93254,26271,BENGALURU,True,1,0,2020-11-03 12:57:30,Explaining India s miraculous Covid recovery T...


In [16]:
# Checking the Sentiment of the Tweet.
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,"Tracking #COVID19 | November 3, 2020 (6:30 pm,...",The Times Of India,610027,13204586,New Delhi,True,26,1,2020-11-03 13:08:23,Tracking November 3 2020 6 30 pm IST India s t...,Positive
1,Daily COVID 19 Cases &amp; Deaths Decline In I...,Mathrubhumi News,46861,50696,"Kerala, India",True,1,0,2020-11-03 13:03:02,Daily COVID 19 Cases amp Deaths Decline In India,Neutral
2,"Active #COVID__19 cases in India — 539,431\n\n...",ORF,156349,80297,India,True,5,3,2020-11-03 13:00:42,Active 19 cases in India 539 431,Negative
3,COVID 19 cases in India until : 03/11/2020 18:...,Coronavirus Bot 🇮🇳,2862,116,,False,0,0,2020-11-03 13:00:00,COVID 19 cases in India until 03 11 2020 18 30...,Negative
4,Explaining India's miraculous Covid recovery\n...,NEWS9,93254,26271,BENGALURU,True,1,0,2020-11-03 12:57:30,Explaining India s miraculous Covid recovery T...,Positive


In [17]:
# Checking the Particular Tweet.

In [18]:
n = 0
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Tracking #COVID19 | November 3, 2020 (6:30 pm, IST)

India's total cases reach 8,267,623

For complete coverage on… https://t.co/8dcsgh5NoC

Clean tweet:
 Tracking November 3 2020 6 30 pm IST India s total cases reach 8 267 623 For complete coverage on

Sentiment of the tweet:
 Positive


In [20]:
n = 1550
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Covid-19 deaths aren't rising as fast in Europe and US, despite soaring new infections. That doesn't mean the virus… https://t.co/msGJM7S281

Clean tweet:
 Covid 19 deaths aren t rising as fast in Europe and US despite soaring new infections That doesn t mean the virus

Sentiment of the tweet:
 Positive


In [21]:
n = 1200
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 COVID 19 cases in India until : 30/10/2020 09:30

Total Cases: 8088046 [+0]😅
Active: 593617🖖🏽
Recovered: 7371898 [+… https://t.co/MOkKHIZm8R

Clean tweet:
 COVID 19 cases in India until 30 10 2020 09 30 Total Cases 8088046 0 Active 593617 Recovered 7371898

Sentiment of the tweet:
 Negative


In [22]:
print("Tweets with Positive Sentiment:",df[df.Sentiment == 'Positive'].shape[0])
print("Tweets with Neutral Sentiment:",df[df.Sentiment == 'Neutral'].shape[0])
print("Tweets with Negative Sentiment:",df[df.Sentiment == 'Negative'].shape[0])

Tweets with Positive Sentiment: 963
Tweets with Neutral Sentiment: 553
Tweets with Negative Sentiment: 484
